In [1]:
import numpy as np
from itertools import product
from scipy.signal import fftconvolve, convolve

In [2]:
def max_pool(x, pool_shape, stride, pooling_axes, backprop=False, dout=None):
    """ Pool the values of an ndarray, by taking the max over a specified pooling
        filter volume that rasters across the specified axes of x with a given stride.

        A backprop flag can be toggled to, instead, perform back-propagation through the
        maxpool layer (i.e. pass gradient values through of the array elements that
        contributed to the forward pooling).

        Parameters
        ----------
        x : numpy.ndarray
            Input array to be pooled.
        pool_shape : Iterable[int, ...]
            Shape of pooling filter.
        stride : int ( > 0)
            Step size used while rastering the pooling filter across x.
        pooling_axes : Union[int, Iterable[int, ...]]
            The axes along which the values of x will be max-pooled.
        backprop : bool, optional
            Indicates whether or not max_pool performs back propagation
            instead of pooling.
        dout : Union[NoneType, numpy.ndarray]
            "Upstream" array, whose values will be back propagated through
             the max-pool layer. This must be specified if backprop is True.

        Returns
        -------
        if backprop is False
            out : numpy.ndarray
                An array of the max-pooled values of x.

        if backprop is True
            dx : numpy.ndarray (shape=x.shape)
                An array of values from dout back-propagated through the pooling layer.
        """

    if type(pooling_axes) is int:
        pooling_axes = (pooling_axes)
    pooling_axes = tuple(pooling_axes)
    
    assert len(pooling_axes) == len(pool_shape)
    pool_only_slice = tuple(slice(None, None) if i in pooling_axes else 0 for i in range(x.ndim))
    outshape = get_outshape(x[pool_only_slice].shape, pool_shape, stride)

    if backprop:
        assert dout is not None, "dout must be provided during backprop"
        mask_view = tuple(np.newaxis if i in pooling_axes else slice(None, None) for i in range(x.ndim))
        dx = np.zeros_like(x, dtype=x.dtype)

    else:
        tmp_shape = list(x.shape)
        for i, ax in enumerate(pooling_axes):
            tmp_shape[ax] = outshape[i]
        out = np.zeros(tmp_shape, dtype=x.dtype)

    all_slices = [slice(None, None) for i in range(x.ndim)]

    # iterate over positions to place the pooling filter
    for i, pos in enumerate(product(*[stride*np.arange(i) for i in outshape])):

        slices = all_slices[:]
        # generate slices to make pooling filter views of x
        for j, start in enumerate(pos):
            slices[pooling_axes[j]] = slice(start, start + pool_shape[j])
        slices = tuple(slices)

        # generate slices of output array to update
        inds = np.unravel_index(i, outshape)
        loc = all_slices[:]
        for cnt, ax in enumerate(pooling_axes):
            loc[ax] = inds[cnt]

        maxes = np.amax(x[slices], axis=pooling_axes)

        if not backprop:
            out[loc] = maxes
        else:
            dx[slices][np.where(x[slices] == maxes[mask_view])] = dout[loc].flat

    if not backprop:
        return out
    else:
        return dx

In [3]:
def get_outshape(dat_shape, kernel_shape, stride):
    """ Returns the shape of the ndarray resulting from the convolution, using specified stride,
        of two ndarrays whose shapes are specified.

        Parameters
        ----------
        dat_shape : Iterable[int, ...]
            Shape of array to be convolved over.
        kernel_shape : Iterable[int, ...]
            Shape of kernel used to perform convolution.
        stride : int ( > 0)
            Step size used while sliding the kernel during the convolution.

        Returns
        -------
        out : numpy.ndarray([int, ...])
            Shape of the array resulting from the convolution."""
    dat_shape = np.array(dat_shape)
    kernel_shape = np.array(kernel_shape)

    assert stride > 0
    stride = int(round(stride))
    assert len(dat_shape) == len(kernel_shape), "kernel and data must have same number of dimensions"

    outshape = (dat_shape-kernel_shape)/stride+1.
    for num in outshape:
        assert num.is_integer(), num
    outshape = np.round(outshape).astype(int)

    return outshape

In [4]:
def flip_ndarray(x):
    """ Reverse the positions of the entries of the input array along all of its axes.

        Parameters
        ----------
        x : numpy.ndarray

        Returns
        -------
        out : numpy.ndarray
            A view of x with all of its axes reversed. Since a view is returned, this operation is O(1).

        Example
        ------
        x = array([[1, 2, 3],
                   [6, 5, 6]])
        flip_ndarray(x))
        >> array([[6, 5, 4],
                  [3, 2, 1]])"""
    loc = tuple(slice(None, None, -1) for i in xrange(x.ndim))
    return x[loc]


In [5]:
def padder(dat, pad, skip_axes=[0]):
    """ Returns an array padded with zeros with specified depth on both sides of each axis. A list of
        axes can be specified, which will not receive any padding.

        Parameters
        ----------
        dat : numpy.ndarray
            Array to be padded
        pad : int ( >= 0)
            Padding depth to be used on each end of a padding axis.
        skip_axes : Union[int, Iterable[int, ...]]
            The indices corresponding to axes that will not be padded.

        Returns
        -------
        out : numpy.ndarray
            Array padded with zeros."""
    assert pad >= 0 and type(pad) == int
    if pad == 0:
        return dat

    if type(skip_axes) == int:
        skip_axes = [skip_axes]
    assert hasattr(skip_axes, '__iter__')
    padding = [(pad, pad) for i in xrange(dat.ndim)]

    for ax in skip_axes:
        padding[ax] = (0, 0)

    return np.pad(dat, padding, mode='constant').astype(dat.dtype)

In [6]:
def my_convd(dat, raster_kernel, stride, outshape=None):
    """ Perform a dot-product convolution of a n-dim kernel with n-dim data"""
    if outshape is None:
        outshape = get_outshape(dat.shape, raster_kernel.shape, stride)
    out = np.zeros(outshape, dtype=dat.dtype)
    indices = range(dat.ndim)
    # iterate over positions to place the kernel
    for i,x in enumerate(product(*[stride*np.arange(i) for i in outshape])):
        # generate slices to make pooling kernel views of x
        slices = tuple(slice(start, start+raster_kernel.shape[j]) for j,start in enumerate(x))
        loc = np.unravel_index(i, outshape)
        out[loc] = np.einsum(dat[slices], indices, raster_kernel, indices)

    return out

In [7]:
dat = np.arange(2*3*4).reshape(2,3,4)

In [8]:
dat

array([[[ 0,  1,  2,  3],
        [ 4,  5,  6,  7],
        [ 8,  9, 10, 11]],

       [[12, 13, 14, 15],
        [16, 17, 18, 19],
        [20, 21, 22, 23]]])

In [13]:
pool_shape = (3,2)
pool_axes  = (1,2)
stride = 2

In [14]:
max_pool(dat, pool_shape, stride, pool_axes)

array([[[ 9, 11]],

       [[21, 23]]])

# Junk

In [ ]:
def back_prop_dw(data, dout, kernel, stride, pad):
    dw = np.zeros_like(kernel, dtype=kernel.dtype)
    for n, dat in enumerate(data):
        dy = dout[n][np.newaxis, :, :]
        if n == 0:  # initialize output and stride information
            npad = np.array([0]+[pad for i in xrange(dat.ndim-1)])
            outshape = (np.array(dat.shape)-np.array(kernel.shape)+2.*npad)/float(stride)+1.
            outshape = np.round(outshape).astype(int)

            if pad:
                pad = [(0, 0)] + [(pad, pad) for i in xrange(dat.ndim-1)]

        all_pos = list(product(*[stride*np.arange(i) for i in outshape]))
        all_slices = [tuple(slice(start, start+kernel.shape[i]) for i,start in enumerate(x)) for x in all_pos]

        if pad:
            dat = np.pad(dat, pad, mode='constant').astype(dat.dtype)

        for i,slices in enumerate(all_slices):
            loc = np.unravel_index(i, outshape)
            dw += dy[loc]*dat[slices]
    return dw


def back_prop_dx(data, dout, kernel, stride, pad):
    for n, dat in enumerate(data):
        dy = dout[n][np.newaxis, :, :]
        out = np.zeros((dat.shape[0], dat.shape[1]+2*pad, dat.shape[2]+2*pad), dtype=data.dtype)
        if n == 0:  # initialize output and stride information
            npad = np.array([0]+[pad for i in xrange(dat.ndim-1)])
            outshape = (np.array(dat.shape)-np.array(kernel.shape)+2.*npad)/float(stride)+1.
            outshape = np.round(outshape).astype(int)

            total_out = np.zeros(data.shape, dtype=dat.dtype)
            all_pos = list(product(*[stride*np.arange(i) for i in outshape]))
            all_slices = [tuple(slice(start, start+kernel.shape[i]) for i,start in enumerate(x)) for x in all_pos]

        for i, slices in enumerate(all_slices):
            loc = np.unravel_index(i, outshape)

            out[slices] += dy[loc]*kernel

        if pad:
            out = out[:, pad:-pad, pad:-pad]

        total_out[n] = out[:]
    return total_out


In [ ]:
def multi_layer_conv(data, kernel, stride, pad, flip_kernel=True, outshape=None, channels=True):
    """ Perform a convolution on a collection of ndarrays of uniform shape with a kernel,
        using a specified stride and padding.

        The convolution

        Parameters
        ----------
        data : Iterable[numpy.ndarray, ...]
            Collection of arrays to be convolved over.
        kernel : numpy.ndarray
            Kernel used to perform convolution.
        stride : int ( > 0)
            Step size used while sliding the kernel during the convolution.
        pad : int ( >= 0)
            Specifies the number of zeroes added to every side of each input array.
        flip_kernel : bool, optional
            When True (default), reverse all of the axes of the kernel before performing convolutions.
        outshape : Union[NoneType, Tuple[int, ...]], optional
            Provide the known output shape of the convolution, allowing the function to bypass
            sanity checks and some initial computations.

        Returns
        -------
        out : numpy.ndarray, shape = (len(data), data[0].shape[0], data[0].shape[1], ...)
            An array of the resulting convolutions. The first axis runs over the results of
            the independent convolutions that were performed.
        """

    if flip_kernel:
        kernel = flip_ndarray(kernel)
    for n, dat in enumerate(data):
        if n == 0:  # initialize output and stride information
            if np.max(dat.shape) >= 500:
                conv = fftconvolve  # use fft method for large input arrays
            else:
                conv = convolve

            if outshape is None:
                npad = np.array([0]+[pad for i in xrange(dat.ndim-1)])
                outshape = (np.array(dat.shape)-np.array(kernel.shape)+2.*npad)/float(stride)+1.
                for num in outshape:
                    assert (num).is_integer(), num
                outshape = np.round(outshape).astype(int)

            if pad:
                if channels:
                    pad = [(0, 0)] + [(pad, pad) for i in xrange(dat.ndim-1)]

            total_out_shape = tuple([data.shape[0]]+[i for i in outshape])
            total_out = np.zeros(total_out_shape, dtype=dat.dtype)

            all_pos = zip(*product(*(stride*np.arange(i) for i in outshape)))

        if pad:
            dat = np.pad(dat, pad, mode='constant').astype(dat.dtype)

        full_conv = conv(dat, kernel, mode='valid')
        if stride == 1:
            total_out[n] = full_conv
        else:
            out = np.zeros(outshape, dtype=full_conv.dtype)
            out.flat = full_conv[all_pos]
            total_out[n] = out
    return total_out

for n, kernel in enumerate(w):
    out[:, n:n+1, :, :] = multi_layer_conv(x, kernel, conv_param['stride'], conv_param['pad'])
    out[:, n:n+1, :, :] += b[n]